In [1]:
# -*- coding: utf-8 -*-
import time
import os
import tensorflow as tf
import numpy as np
%run Bi_LSTM.py
%run Word2Vec.py
import pandas as pd

c:\users\pjk\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# 여기부터 하면 됨

In [2]:
from tqdm import tqdm_notebook
from eunjeon import Mecab
from konlpy.tag import Okt

In [3]:
mecab = Mecab()
okt = Okt()
W2V = Word2Vec()

In [4]:
df = pd.read_csv('../data/자소서_균등3.csv',index_col=0, engine='python', encoding='utf8')

## 데이터 섞기

In [5]:
df = df.reset_index().drop(columns={'index'})

In [6]:
rows = np.random.choice(df.index.values, len(df), replace=False)
sampled_df = df.loc[rows]

In [7]:
df = sampled_df

In [8]:
dataset = int(len(df) * 0.7)

In [9]:
df.iloc[:dataset].groupby('label').agg('count')

,c_origin,category,jss
label,,,
0.0,2781,3350,3350
1.0,2351,2630,2630
2.0,2184,2758,2758


In [10]:
df.head()

,c_origin,category,jss,label
9587,서비스,기타,회계학원을 운영하시던 친구 어머니 추천으로 약국 사무실에서 경리 회계 아르바이트를 ...,0.0
3758,회계·재무·세무·IR,재무·금융,"아무래도 군인이셨던 아버지의 영향으로 제 성격이 만들어진 것 같습니다.\n우선, 말...",1.0
10314,기획·전략·경영,기타,제가 회사를 선택하는 기준은 실력과 가치입니다. 먼저 실력은 해당 분야에서 최고의 ...,0.0
4241,증권·투자·펀드·외환,재무·금융,[500명의 마음을 움직인 우수인턴 도전기]지난해 은행 인턴으로 근무하며 모교에서 ...,1.0
4893,금융사무,재무·금융,[사법고시를 준비하며 얻은 경험]\n저는 작년까지 대학에서 다양한 법 전공 수업을 ...,1.0


In [11]:
data = []
for txt in tqdm_notebook(df.jss):
    data.append(mecab.morphs(txt))

In [12]:
labels = df.label

# 워드 임베딩

In [13]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
tf.reset_default_graph()
W2V = Word2Vec()

In [14]:
dataset = int(len(data)*0.8)

In [15]:
# 70% train, 30% test
train_X = data[:dataset]
train_Y = labels[:dataset]
test_X = data[dataset:]
test_Y = labels[dataset:]

In [16]:
#nb_classes = len(df.category.unique())

In [17]:
nb_classes = len(df.label.unique())
nb_classes

3

In [18]:
train_X_ = W2V.Convert2Vec("../data/word2vec_mecab_morphs2.model",train_X)
test_X_ = W2V.Convert2Vec("../data/word2vec_mecab_morphs2.model",test_X)

In [19]:
train_Y_ = W2V.One_hot(train_Y)
test_Y_ = W2V.One_hot(test_Y)

# 파라미터

In [20]:
Batch_size = 32
Total_size = len(train_X)
Vector_size = 300 # 모델의 워드벡터 사이즈
train_seq_length = [len(x) for x in train_X]
test_seq_length = [len(x) for x in test_X]
Maxseq_length = max(max(train_seq_length), max(test_seq_length))
learning_rate = 0.001
lstm_units = 128
num_class = nb_classes
training_epochs = 3
print(Maxseq_length)

2467


# 그래프 만들기

In [21]:
X = tf.placeholder(tf.float32, shape = [None, Maxseq_length, Vector_size], name = 'X')
Y = tf.placeholder(tf.float32, shape = [None, num_class], name = 'Y')
seq_len = tf.placeholder(tf.int32, shape = [None])
keep_prob = tf.placeholder(tf.float32, shape = None)

In [22]:
BiLSTM = Bi_LSTM(lstm_units, num_class, keep_prob)

In [23]:
with tf.variable_scope("loss", reuse = tf.AUTO_REUSE):
    logits = BiLSTM.logits(X, BiLSTM.W, BiLSTM.b, seq_len)
    loss, optimizer = BiLSTM.model_build(logits, Y, learning_rate)
prediction = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead


In [24]:
total_batch = int(len(train_X) / Batch_size)
test_batch = int(len(test_X) / Batch_size)

In [25]:
# modelName = "./BiLSTM_mecab_morphs_eqaul_4/BiLSTM_model.ckpt"
# saver = tf.train.Saver()

# train_acc = []
# train_loss = []
# test_acc = []
# test_loss = []

In [26]:
# 모델 불러오기
saver = tf.train.Saver()
init = tf.global_variables_initializer()
modelName = "BiLSTM_mecab_morphs_eqaul_4/BiLSTM_model.ckpt"

train_acc = []
train_loss = []
test_acc = []
test_loss = []

sess = tf.Session(config=config)
sess.run(init)
start_time = time.time()
saver.restore(sess, modelName)
train_writer = tf.summary.FileWriter('./BiLSTM_mecab_morphs_eqaul_4', sess.graph)
merged = BiLSTM.graph_build()

# 세션 런

In [27]:
# sess = tf.Session(config=config)
# init = tf.global_variables_initializer()
# start_time = time.time()
# sess.run(init)
# train_writer = tf.summary.FileWriter('./BiLSTM_mecab_morphs_eqaul_4', sess.graph)
# merged = BiLSTM.graph_build()

In [28]:
for epoch in range(training_epochs):  
#for epoch in range(5,8):

    avg_acc, avg_loss = 0. , 0.
    mask = np.random.permutation(len(train_X_))
    train_X_ = train_X_[mask]
    train_Y_ = train_Y_[mask]

    for step in tqdm_notebook(range(total_batch)):

        train_batch_X = train_X_[step*Batch_size : step*Batch_size+Batch_size]
        train_batch_Y = train_Y_[step*Batch_size : step*Batch_size+Batch_size]
        batch_seq_length = train_seq_length[step*Batch_size : step*Batch_size+Batch_size]

        train_batch_X = W2V.Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size)

        sess.run(optimizer, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
        # Compute average loss
        loss_ = sess.run(loss, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length,
                                          keep_prob : 0.75})
        avg_loss += loss_ / total_batch

        acc = sess.run(accuracy , feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length,
                                             keep_prob : 0.75})
        avg_acc += acc / total_batch
        print("epoch : {:02d} step : {:04d} loss = {:.6f} accuracy= {:.6f}".format(epoch+1, step+1, loss_, acc))

    summary = sess.run(merged, feed_dict = {BiLSTM.loss : avg_loss, BiLSTM.acc : avg_acc})       
    train_writer.add_summary(summary, epoch)

    t_avg_acc, t_avg_loss = 0. , 0.
    print(avg_loss, avg_acc)
    print("Test cases, could take few minutes")
    
    for step in tqdm_notebook(range(test_batch)):

        test_batch_X = test_X_[step*Batch_size : step*Batch_size+Batch_size]
        test_batch_Y = test_Y_[step*Batch_size : step*Batch_size+Batch_size]
        batch_seq_length = test_seq_length[step*Batch_size : step*Batch_size+Batch_size]

        test_batch_X = W2V.Zero_padding(test_batch_X, Batch_size, Maxseq_length, Vector_size)

        # Compute average loss
        loss2 = sess.run(loss, feed_dict={X: test_batch_X, Y: test_batch_Y, seq_len: batch_seq_length,
                                          keep_prob : 1.0})
        t_avg_loss += loss2 / test_batch

        t_acc = sess.run(accuracy , feed_dict={X: test_batch_X, Y: test_batch_Y, seq_len: batch_seq_length,
                                               keep_prob : 1.0})
        t_avg_acc += t_acc / test_batch

    print("<Train> Loss = {:.6f} Accuracy = {:.6f}".format(avg_loss, avg_acc))
    print("<Test> Loss = {:.6f} Accuracy = {:.6f}".format(t_avg_loss, t_avg_acc))
    train_loss.append(avg_loss)
    train_acc.append(avg_acc)
    test_loss.append(t_avg_loss)
    test_acc.append(t_avg_acc)

epoch : 01 step : 0001 loss = 0.702122 accuracy= 0.812500
epoch : 01 step : 0002 loss = 0.877484 accuracy= 0.687500
epoch : 01 step : 0003 loss = 0.996040 accuracy= 0.468750
epoch : 01 step : 0004 loss = 0.999193 accuracy= 0.531250
epoch : 01 step : 0005 loss = 1.046489 accuracy= 0.468750
epoch : 01 step : 0006 loss = 1.023164 accuracy= 0.468750
epoch : 01 step : 0007 loss = 1.019504 accuracy= 0.437500
epoch : 01 step : 0008 loss = 1.009714 accuracy= 0.500000
epoch : 01 step : 0009 loss = 0.955890 accuracy= 0.593750
epoch : 01 step : 0010 loss = 0.836168 accuracy= 0.687500
epoch : 01 step : 0011 loss = 1.018399 accuracy= 0.500000
epoch : 01 step : 0012 loss = 1.027949 accuracy= 0.406250
epoch : 01 step : 0013 loss = 1.102132 accuracy= 0.375000
epoch : 01 step : 0014 loss = 1.025871 accuracy= 0.593750
epoch : 01 step : 0015 loss = 0.863594 accuracy= 0.625000
epoch : 01 step : 0016 loss = 0.874030 accuracy= 0.625000
epoch : 01 step : 0017 loss = 1.094737 accuracy= 0.500000
epoch : 01 ste

epoch : 01 step : 0142 loss = 0.729536 accuracy= 0.718750
epoch : 01 step : 0143 loss = 0.878878 accuracy= 0.562500
epoch : 01 step : 0144 loss = 0.811811 accuracy= 0.625000
epoch : 01 step : 0145 loss = 0.870247 accuracy= 0.656250
epoch : 01 step : 0146 loss = 0.667542 accuracy= 0.687500
epoch : 01 step : 0147 loss = 0.792864 accuracy= 0.625000
epoch : 01 step : 0148 loss = 0.839172 accuracy= 0.625000
epoch : 01 step : 0149 loss = 0.783320 accuracy= 0.625000
epoch : 01 step : 0150 loss = 0.967589 accuracy= 0.500000
epoch : 01 step : 0151 loss = 0.811916 accuracy= 0.687500
epoch : 01 step : 0152 loss = 0.974714 accuracy= 0.375000
epoch : 01 step : 0153 loss = 0.950967 accuracy= 0.406250
epoch : 01 step : 0154 loss = 0.976485 accuracy= 0.437500
epoch : 01 step : 0155 loss = 0.696646 accuracy= 0.781250
epoch : 01 step : 0156 loss = 0.821623 accuracy= 0.593750
epoch : 01 step : 0157 loss = 0.849553 accuracy= 0.593750
epoch : 01 step : 0158 loss = 0.887217 accuracy= 0.531250
epoch : 01 ste

epoch : 01 step : 0283 loss = 0.705713 accuracy= 0.625000
epoch : 01 step : 0284 loss = 0.739680 accuracy= 0.687500
epoch : 01 step : 0285 loss = 0.733502 accuracy= 0.656250
epoch : 01 step : 0286 loss = 0.707503 accuracy= 0.593750
epoch : 01 step : 0287 loss = 0.623350 accuracy= 0.718750
epoch : 01 step : 0288 loss = 0.750233 accuracy= 0.531250
epoch : 01 step : 0289 loss = 0.740717 accuracy= 0.718750
epoch : 01 step : 0290 loss = 0.759747 accuracy= 0.687500
epoch : 01 step : 0291 loss = 0.651654 accuracy= 0.718750
epoch : 01 step : 0292 loss = 0.614630 accuracy= 0.718750
epoch : 01 step : 0293 loss = 0.943416 accuracy= 0.593750
epoch : 01 step : 0294 loss = 0.732376 accuracy= 0.718750
epoch : 01 step : 0295 loss = 0.724400 accuracy= 0.625000
epoch : 01 step : 0296 loss = 0.868955 accuracy= 0.687500
epoch : 01 step : 0297 loss = 0.665245 accuracy= 0.750000
epoch : 01 step : 0298 loss = 0.742134 accuracy= 0.687500
epoch : 01 step : 0299 loss = 0.809211 accuracy= 0.593750
epoch : 01 ste


<Train> Loss = 0.828517 Accuracy = 0.611178
<Test> Loss = 0.798312 Accuracy = 0.629808


epoch : 02 step : 0001 loss = 0.577313 accuracy= 0.718750
epoch : 02 step : 0002 loss = 0.646493 accuracy= 0.718750
epoch : 02 step : 0003 loss = 0.584702 accuracy= 0.843750
epoch : 02 step : 0004 loss = 0.629436 accuracy= 0.718750
epoch : 02 step : 0005 loss = 0.534374 accuracy= 0.875000
epoch : 02 step : 0006 loss = 0.629063 accuracy= 0.843750
epoch : 02 step : 0007 loss = 0.627783 accuracy= 0.718750
epoch : 02 step : 0008 loss = 0.606395 accuracy= 0.750000
epoch : 02 step : 0009 loss = 0.540711 accuracy= 0.750000
epoch : 02 step : 0010 loss = 0.563252 accuracy= 0.875000
epoch : 02 step : 0011 loss = 0.571064 accuracy= 0.812500
epoch : 02 step : 0012 loss = 0.642110 accuracy= 0.687500
epoch : 02 step : 0013 loss = 0.683332 accuracy= 0.656250
epoch : 02 step : 0014 loss = 0.615902 accuracy= 0.781250
epoch : 02 step : 0015 loss = 0.767466 accuracy= 0.656250
epoch : 02 step : 0016 loss = 0.648490 accuracy= 0.750000
epoch : 02 step : 0017 loss = 0.636051 accuracy= 0.718750
epoch : 02 ste

...


<Train> Loss = 0.630348 Accuracy = 0.734275
<Test> Loss = 0.711389 Accuracy = 0.679888


epoch : 03 step : 0001 loss = 0.437482 accuracy= 0.875000
epoch : 03 step : 0002 loss = 0.461329 accuracy= 0.875000
epoch : 03 step : 0003 loss = 0.852118 accuracy= 0.593750
epoch : 03 step : 0004 loss = 0.843973 accuracy= 0.687500
epoch : 03 step : 0005 loss = 0.574860 accuracy= 0.812500
epoch : 03 step : 0006 loss = 0.438468 accuracy= 0.875000
epoch : 03 step : 0007 loss = 0.430624 accuracy= 0.906250
epoch : 03 step : 0008 loss = 0.617286 accuracy= 0.718750
epoch : 03 step : 0009 loss = 0.541671 accuracy= 0.812500
epoch : 03 step : 0010 loss = 0.517042 accuracy= 0.843750
epoch : 03 step : 0011 loss = 0.488508 accuracy= 0.812500
epoch : 03 step : 0012 loss = 0.708998 accuracy= 0.750000
epoch : 03 step : 0013 loss = 0.510992 accuracy= 0.875000
epoch : 03 step : 0014 loss = 0.389584 accuracy= 0.937500
epoch : 03 step : 0015 loss = 0.392198 accuracy= 0.843750
epoch : 03 step : 0016 loss = 0.610589 accuracy= 0.687500
epoch : 03 step : 0017 loss = 0.478128 accuracy= 0.906250
epoch : 03 ste

epoch : 03 step : 0142 loss = 0.409826 accuracy= 0.875000
epoch : 03 step : 0143 loss = 0.419791 accuracy= 0.843750
epoch : 03 step : 0144 loss = 0.453641 accuracy= 0.843750
epoch : 03 step : 0145 loss = 0.604993 accuracy= 0.718750
epoch : 03 step : 0146 loss = 0.375901 accuracy= 0.875000
epoch : 03 step : 0147 loss = 0.408753 accuracy= 0.843750
epoch : 03 step : 0148 loss = 0.485068 accuracy= 0.843750
epoch : 03 step : 0149 loss = 0.443143 accuracy= 0.812500
epoch : 03 step : 0150 loss = 0.458853 accuracy= 0.750000
epoch : 03 step : 0151 loss = 0.328581 accuracy= 0.812500
epoch : 03 step : 0152 loss = 0.479130 accuracy= 0.812500
epoch : 03 step : 0153 loss = 0.502784 accuracy= 0.781250
epoch : 03 step : 0154 loss = 0.474934 accuracy= 0.750000
epoch : 03 step : 0155 loss = 0.614565 accuracy= 0.812500
epoch : 03 step : 0156 loss = 0.340861 accuracy= 0.843750
epoch : 03 step : 0157 loss = 0.432377 accuracy= 0.843750
epoch : 03 step : 0158 loss = 0.549546 accuracy= 0.750000
epoch : 03 ste

epoch : 03 step : 0283 loss = 0.458373 accuracy= 0.906250
epoch : 03 step : 0284 loss = 0.644806 accuracy= 0.718750
epoch : 03 step : 0285 loss = 0.469638 accuracy= 0.843750
epoch : 03 step : 0286 loss = 0.631455 accuracy= 0.718750
epoch : 03 step : 0287 loss = 0.447190 accuracy= 0.812500
epoch : 03 step : 0288 loss = 0.455017 accuracy= 0.875000
epoch : 03 step : 0289 loss = 0.410518 accuracy= 0.875000
epoch : 03 step : 0290 loss = 0.630311 accuracy= 0.718750
epoch : 03 step : 0291 loss = 0.582179 accuracy= 0.750000
epoch : 03 step : 0292 loss = 0.461729 accuracy= 0.781250
epoch : 03 step : 0293 loss = 0.382129 accuracy= 0.875000
epoch : 03 step : 0294 loss = 0.344689 accuracy= 0.875000
epoch : 03 step : 0295 loss = 0.551937 accuracy= 0.750000
epoch : 03 step : 0296 loss = 0.417318 accuracy= 0.843750
epoch : 03 step : 0297 loss = 0.547242 accuracy= 0.843750
epoch : 03 step : 0298 loss = 0.419194 accuracy= 0.875000
epoch : 03 step : 0299 loss = 0.604853 accuracy= 0.750000
epoch : 03 ste


<Train> Loss = 0.489778 Accuracy = 0.810998
<Test> Loss = 0.724620 Accuracy = 0.683894


In [29]:
train_loss = pd.DataFrame({"train_loss":train_loss})
train_acc = pd.DataFrame({"train_acc":train_acc})
test_loss = pd.DataFrame({"test_loss":test_loss})
test_acc = pd.DataFrame({"test_acc":test_acc})
df = pd.concat([train_loss,train_acc,test_loss,test_acc], axis = 1)
df.to_csv("./BiLSTM_mecab_morphs_eqaul_4/loss_accuracy2.csv", sep =",", index=False)

train_writer.close()
duration = time.time() - start_time
minute = int(duration / 60)
second = int(duration) % 60
print("%dminutes %dseconds" % (minute,second))
save_path = saver.save(sess, modelName)

print ('save_path',save_path)

49minutes 14seconds
save_path ./BiLSTM_mecab_morphs_eqaul_4/BiLSTM_model.ckpt


# 에러나는 거 지우기..

In [ ]:
lis = []
for i in range(len(data)):
    if len(data[i]) == 0:
        lis.append(i)

In [ ]:
df.index

In [ ]:
bugidx = []
for i in lis:
    bugidx.append(df.index[i])

In [ ]:
df.drop(bugidx,inplace=True)

In [ ]:
df.head()